In [5]:
import dask.dataframe as dd
import pandas as pd
from huggingface_hub import hf_hub_download
import tqdm as notebook_tqdm
import matplotlib.pyplot as plt

# Исходные файлы загружены на ресурс https://huggingface.co/datasets
REPO_ID = "DmitriiMiptEdu/event_data"
adv = 'adv_data.csv'
event = 'event_data.csv'
users = 'users_data.csv'


In [2]:
!pip install gcsfs google-auth pandas pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 12.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 59.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 25.5 MB/s eta 0:00:

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

# Функция для аутентификации с Google Drive API
def authenticate_google_drive():
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()  # Открывает браузер для аутентификации
    drive = GoogleDrive(gauth)
    return drive


In [ ]:
# Аутентификация и создание экземпляра Google Drive
drive = authenticate_google_drive()

In [7]:
# event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"), low_memory=False,  blocksize=25e6, dtype={'event_status': 'object','user_phone': 'object'})
event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"), low_memory=False,  dtype={'event_status': 'object','user_phone': 'object'})
adv_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=adv, repo_type="dataset"), low_memory=False, )
users_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=users, repo_type="dataset"),low_memory=False,  dtype={'registration_dtm': 'object', 'delete_dtm': 'object','user_phone': 'object'} )
print(f'\nevent_data.info():\n {event_data.info()}')
print(f'\n\nadv_data.info():\n {adv_data.info()}')
print(f'\n\nusers_data.info():\n {users_data.info()}')

<class 'dask_expr.DataFrame'>
Columns: 9 entries, id to product_type
dtypes: float64(1), string(8)
event_data.info():
 None
<class 'dask_expr.DataFrame'>
Columns: 9 entries, user_id to campaign_cost
dtypes: float64(1), string(8)

adv_data.info():
 None
<class 'dask_expr.DataFrame'>
Columns: 4 entries, user_phone to delete_dtm
dtypes: string(4)

users_data.info():
 None


In [8]:
# Предобработка данных
event_data['user_phone'] = event_data['user_phone'].str.replace('+', '')
users_data['user_phone'] = users_data['user_phone'].str.replace('+', '')

In [19]:
event_data.head(20)

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
0,1263769bddda6eaaf7c8b8b22812ad6a063ad234d80782...,79140092454,show,2023-10-05 12:03:04.125806,NaN,NaT,NaN,RUB,mortgage
1,ae05865023d28ae72dd17955851263ead2d4daf11c69e0...,79125904946,show,2023-05-16 10:16:54.554523,NaN,NaT,NaN,RUB,loan
2,9861b00e9c9a99a6fdb2e5c11beebe9581eb1dda36ddc4...,79184556300,show,2023-12-23 22:46:44.066599,NaN,NaT,NaN,RUB,loan
3,349366af4823add107077299482f21031941fc6d4cd571...,79119287528,show,2023-07-30 05:25:46.285514,NaN,NaT,NaN,RUB,loan
4,2be2933460f58bb2d9a311ede6bb6ee064989568ab57b8...,79119287528,click,2023-07-30 05:25:46.285514,NaN,2023-07-30 05:42:59.014970,2.18,USD,loan
5,f22ca540ad7f36fa540a40b93254d4f4a4098db0e54b44...,79159249645,show,2023-08-23 20:38:20.004356,NaN,NaT,NaN,RUB,deposits
6,81e2b1545f71f464613168b2c0dddc9bfa86092120d843...,79120583792,show,2023-05-29 05:41:12.574617,NaN,NaT,NaN,RUB,loan
7,33ee92d770d2b36351fa5cef36710342c2d114ff146f85...,79168803865,show,2023-05-14 04:06:07.519296,NaN,NaT,NaN,RUB,deposits
8,7e60f1cf394c4cc9ec6408f567c99ff0f964d7a9fdf7f7...,79146940847,show,2024-02-01 16:57:06.006143,NaN,NaT,NaN,RUB,deposits
9,6b2ecdddedae517cc297d9b8801ba58dbe417068dbe4a7...,79111263094,show,2023-09-06 08:04:38.091411,NaN,NaT,NaN,RUB,deposits


In [17]:
users_data.head()


,user_phone,user_id,registration_dtm,delete_dtm
0,79100004508,0000038524a303ed5876b823574848e0,2023-01-15 15:33:14.952938,NaT
1,79100005383,000003ef5bdfb2545c8d5ab693142e30,2022-11-21 22:43:54.418792,NaT
2,79100006444,0000056caffa28b60bf17bc69fe4366f,2022-12-23 03:00:04.088219,NaT
3,79100007221,00000a6f236f49c556e9a7b31d1b01e4,2022-12-23 03:31:50.935786,NaT
4,79100009619,00000cbd32277c99f2bd5e180a21f2b2,2023-01-25 14:31:17.234998,NaT


In [5]:
adv_data.head()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,2023-09-18 19:39:25.390618,2023-09-18 19:39:25.390618,finance.rambler.ru / cpm,rebrand_megamaket_reels,app,RUB,NaN
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,2023-04-11 06:47:33.199369,2023-04-11 06:47:33.199369,yandex / organic,adv_retarget_brandkeys,web,RUB,NaN
2,e9e12347e50732804d2f8ded00743a07,show,2023-10-28 04:35:48.329870,2023-10-28 04:35:48.329870,yandex / organic,bk_light_brandkeys,web,RUB,NaN
3,fa0cd363d1978a27a078152cd263c670,show,2023-05-28 18:25:18.694051,2023-05-28 18:25:18.694051,yandex / organic,retarget_mind_response,api,RUB,NaN
4,fa0cd363d1978a27a078152cd263c670,click,2023-05-31 18:55:55.352680,2023-05-31 18:55:55.352680,regular / sms,light_clickarrow_clickarrow,api,USD,1.34


In [9]:
# Преобразование полей датасетов во временные метки
event_data['event_dtm'] = dd.to_datetime(event_data['event_dtm'])
event_data['update_dtm'] = dd.to_datetime(event_data['update_dtm'])
adv_data['campaign_start_dtm'] = dd.to_datetime(adv_data['campaign_start_dtm'])
adv_data['campaign_end_dtm'] = dd.to_datetime(adv_data['campaign_end_dtm'])
users_data['registration_dtm'] = dd.to_datetime(users_data['registration_dtm'])
users_data['delete_dtm'] = dd.to_datetime(users_data['delete_dtm'])


In [10]:
merged_data = dd.merge(event_data, users_data[['user_phone', 'user_id']], on='user_phone',how='left')


In [7]:
# Объединение users_data и event_data
merged_data = dd.merge(event_data, users_data, on='user_phone', how='left')

In [11]:
merged_data.head(10)

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type,user_id
0,d2044309c979bdd2db28f4262680e2e9ffe178391e919c...,79193491698,show,2023-03-22 20:16:26.631412,NaN,NaT,NaN,RUB,loan,ef799f44dfceddf39b84114427ceb812
1,4be71e55358963f0dbadfc3b92a46cc2b3e7d2327ec407...,79193491698,click,2023-03-22 20:16:26.631412,NaN,2023-03-22 20:28:50.420748,0.0,RUB,loan,ef799f44dfceddf39b84114427ceb812
2,d8a7a7de39eb650205019e36c2fa08fe725ec0ec638e31...,79125615391,show,2023-10-23 02:44:25.899221,NaN,NaT,NaN,RUB,deposits,42de9085f7bd61cc40d93579dfd06ad5
3,9b398c82baa170f07dc94212ac46825807ec5c467564f5...,79138921423,show,2023-09-18 14:56:32.805804,NaN,NaT,NaN,RUB,loan,64bfe9dd8dac292599bd52f5bace566a
4,eb66a3c5ea393215d00d0c0d66b6a8b35edca595f0c370...,79181912815,show,2023-02-27 17:39:37.166158,NaN,NaT,NaN,RUB,deposits,d213cff79befb1aaf386fbb6a3aedf2d
5,272fa376ec3f2a863042381c9784e6e63abb401dcfd4c2...,79148895108,show,2023-08-28 13:06:58.000170,NaN,NaT,NaN,RUB,loan,7e219ce8ecc690e6688d7c7961a896ad
6,0c4ce70629bf8a707fd96c02ab5e8fafe8f66665bc065c...,79143041274,show,2023-03-14 11:14:28.378652,NaN,NaT,NaN,USD,loan,6f3f4d6d7f8a64013dd4fc321c61e86f
7,46d92a9a82acdfd4609beeb3b150bb97a96334e4ce3885...,79112623012,show,2023-10-15 19:33:51.377989,NaN,NaT,NaN,RUB,loan,21e0ce0ebb60f17aae1c340ce95191f2
8,98aa900b70fff0a9a172f9b18e814945d121e065938d2e...,79112465778,show,2023-12-20 18:26:05.770005,NaN,NaT,NaN,RUB,deposits,217ab7805da6d6570bf339a1acfb0e82
9,0ddae564035cc3b56981f89ac440bbf3e82cdf30af8625...,79134879620,show,2023-06-16 10:24:48.812211,NaN,NaT,NaN,RUB,mortgage,5a6a8bd5553677f1672688b6825c893e


In [12]:

# Объединение с adv_data
merged_data = dd.merge(merged_data, adv_data, on='user_id', how='left')

In [18]:
# Удаление столбца user_phone
merged_data = merged_data.drop(columns=['user_phone'])

In [19]:
merged_data.head(10)

,id,event_type,event_dtm,event_status,update_dtm,payout,currency_x,product_type,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_y,campaign_cost
0,7fa4c261201c2ba5d9ae07050134d5d96691b3515d773a...,show,2023-10-24 05:43:52.765568,NaN,NaT,NaN,RUB,deposits,64c79cbfdffad79a44fa6b5ac35fe4fb,show,2023-07-13 13:21:14.824676,2023-07-13 13:21:14.824676,google / organic,client_light_response,app,RUB,NaN
1,7fa4c261201c2ba5d9ae07050134d5d96691b3515d773a...,show,2023-10-24 05:43:52.765568,NaN,NaT,NaN,RUB,deposits,64c79cbfdffad79a44fa6b5ac35fe4fb,click,2023-05-27 00:49:19.259265,2023-05-27 00:49:19.259265,yandex / organic,brandkeys_bk_rebrand,web,RUB,0.0
2,1973c567bb3fdfcae015c2ffb1a2b074063a850d098544...,show,2023-04-25 23:52:08.917981,NaN,NaT,NaN,RUB,deposits,9cdfad0d9ba96251f56836444559ecd0,show,2023-04-01 14:12:41.589754,2023-04-01 14:12:41.589754,yandex / organic,response_clickarrow_light,web,USD,NaN
3,1973c567bb3fdfcae015c2ffb1a2b074063a850d098544...,show,2023-04-25 23:52:08.917981,NaN,NaT,NaN,RUB,deposits,9cdfad0d9ba96251f56836444559ecd0,show,2023-05-02 23:11:12.765193,2023-05-02 23:11:12.765193,email / email,light_target_market,app,USD,NaN
4,1472937a8d23b4d4c1b94225a3fe31f2873717488ffc4b...,show,2023-05-01 04:01:20.874463,NaN,NaT,NaN,RUB,loan,74cdaa47992f99e6ac8397725a86f230,show,2023-11-05 22:14:32.145226,2023-11-05 22:14:32.145226,google / organic,target_title_target,app,RUB,NaN
5,86525650182836616573d09ad65ae4009fc1802c5b6813...,click,2023-05-01 04:01:20.874463,NaN,2023-05-01 04:18:32.050433,237.0,RUB,loan,74cdaa47992f99e6ac8397725a86f230,show,2023-11-05 22:14:32.145226,2023-11-05 22:14:32.145226,google / organic,target_title_target,app,RUB,NaN
6,1e2478ef22b07ba9dd8cdf4da4a0a333da2de9c0244a53...,show,2023-03-05 23:58:28.654758,NaN,NaT,NaN,RUB,loan,908e5d92e97efdda349f752b4c68b74c,show,2023-07-31 13:06:06.268760,2023-07-31 13:06:06.268760,regular / sms,case_light_megamaket,web,RUB,NaN
7,21497cbbb000b05db2922ba67dd5821b5d61f5cbcb4435...,show,2023-08-22 00:30:15.342207,NaN,NaT,NaN,RUB,loan,9dc9d5284b137d3ed1a2469961477fad,show,2023-11-05 06:30:02.982081,2023-11-05 06:30:02.982081,yandex / organic,target_operator_clickarrow,app,RUB,NaN
8,e534da7233ebe171050fc68f05374134a35398d8b6b672...,show,2023-10-05 05:07:51.449825,NaN,NaT,NaN,RUB,loan,87c8d695eaf2eb5db2858cb4ae87196d,show,2023-02-23 21:12:38.226433,2023-02-23 21:12:38.226433,yandex / cpc,response_adv_brandkeys,app,RUB,NaN
9,e534da7233ebe171050fc68f05374134a35398d8b6b672...,show,2023-10-05 05:07:51.449825,NaN,NaT,NaN,RUB,loan,87c8d695eaf2eb5db2858cb4ae87196d,click,2023-01-26 12:23:44.371803,2023-01-26 12:23:44.371803,yandex / organic,mind_rebrand_operator,web,RUB,0.0


In [20]:
merged_data.info()


<class 'dask_expr.DataFrame'>
Columns: 17 entries, id to campaign_cost
dtypes: datetime64[ns](4), float64(2), string(11)

In [9]:
# Анализ событий пользователей
# Сортировка событий по пользователям и времени
merged_data = merged_data.sort_values(by=['user_id', 'event_dtm'])

In [25]:
output_path = 'merged_data_folder'
merged_data.to_parquet(output_path, engine='pyarrow', compression='snappy')

In [24]:
output_file = 'merged_data.csv'

merged_data.to_csv(output_file, single_file=True, index=False)

['/workspace/magnum_scopum_sravni_ru_dataton_2024/datasets/merged_data.csv']

In [15]:
merged_data_pd.head()

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency_x,product_type,user_id,registration_dtm,delete_dtm,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_y,campaign_cost


# Разработка модели атрибуции

In [14]:
# Реализация модели линейной атрибуции
def linear_attribution(events):
    n = len(events)
    attribution = 1.0 / n
    events['attribution'] = attribution
    return events

# Применение линейной модели атрибуции
#attributed_data = merged_data_pd.groupby('user_phone', group_keys=False).apply(linear_attribution).reset_index(drop=True)
attributed_data = merged_data_pd.groupby('user_phone', group_keys=False).apply(lambda x: linear_attribution(x)).reset_index(drop=True)




/tmp/ipykernel_1450/3458240431.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  attributed_data = merged_data_pd.groupby('user_phone', group_keys=False).apply(lambda x: linear_attribution(x)).reset_index(drop=True)


In [44]:
attributed_data.head()

# Визуализация результатов

In [ ]:
# Сбор данных для визуализации
source_attribution = attributed_data.groupby('source_medium')['attribution'].sum().compute()
print(source_attribution)

# Визуализация результатов
source_attribution_pd = source_attribution.compute()

# Пример простой визуализации распределения атрибуции
source_attribution_pd.plot(kind='bar')
plt.title('Атрибуция по источникам трафика')
plt.xlabel('Источник')
plt.ylabel('Атрибуция')
plt.show()